<a href="https://colab.research.google.com/github/nlzzq/MDNA/blob/main/ComfyUIonColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#`mountUmount(`<font size="3px" color="#01c968">`Gdrive`</font>`)`



In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive!
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

#`Setup And Update ComfyUI`



In [ ]:
from pathlib import Path

OPTIONS = {}

DRIVE_PATH = ""  # @param {type:"string"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = '/content/ComfyUI'
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if DRIVE_PATH:

    WORKSPACE = DRIVE_PATH+"/ComfyUI"
    %cd {DRIVE_PATH}

![ ! -d WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

%cd {WORKSPACE}

# `Models Download`

## Download MODELS

In [ ]:
%cd {WORKSPACE}
# Install Aria2
!apt-get -y install -qq aria2


from google.colab import userdata

CIVITAI_API_TOKEN = userdata.get('CIVITAI_API_TOKEN')
print("Loaded API key:", "✅" if CIVITAI_API_TOKEN else "❌ Not found")

# Install custom nodes

def install_custom_node(url):
  %cd /content/ComfyUI/custom_nodes
  !git clone {url}

def downloadModel(url, filename = None):
  if 'huggingface.co' in url:
    if filename is None:
      filename = url.split('/')[-1]
      filename = filename.removesuffix('?download=true')
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    if filename:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}?token={CIVITAI_API_TOKEN} -o {filename}
    else:
      !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}?token={CIVITAI_API_TOKEN}


# install custom nodes
#install_custom_node('https://github.com/ltdrdata/ComfyUI-Manager.git')
#install_custom_node('https://github.com/ltdrdata/ComfyUI-Impact-Pack')


# download models
%cd ./models/checkpoints

# CyberRealistic Pony
downloadModel('https://civitai.com/api/download/models/2071650')

# Protovision XL
#downloadModel('https://civitai.com/api/download/models/201514')
# Juggernaut XL Hyper
#downloadModel('https://civitai.com/api/download/models/471120')
# Juppernaut X
#downloadModel('https://civitai.com/api/download/models/456194')
# Juggernaut v8
#downloadModel('https://civitai.com/api/download/models/288982')

# Huggingface example
#downloadModel('https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors?download=true')

%cd {WORKSPACE}

## OTHERS

In [ ]:
# !wget -c https://huggingface.co/datasets/StefanFalkok/Wan_2.2_I2V_10steps/resolve/main/Wan_2.2_I2V_HighNoise_10steps_fp8.safetensors -P ./models/diffusion_models/
# !wget -c https://huggingface.co/datasets/StefanFalkok/Wan_2.2_I2V_10steps/resolve/main/Wan_2.2_I2V_LowNoise_10steps_fp8.safetensors -P ./models/diffusion_models/

!wget -c https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/text_encoders/umt5_xxl_fp16.safetensors -P ./models/text_encoders/
# 使用 wget 下载，并检查文件大小
# !wget -L "https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/vae/wan2.2_vae.safetensors" -O /content/ComfyUI/models/vae/wan_2.2_vae.safetensors
# VAE
# !wget -c https://huggingface.co/Comfy-Org/Wan_2.2_ComfyUI_Repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors -P ./models/vae/

--2026-02-25 16:42:21--  https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/text_encoders/umt5_xxl_fp16.safetensors
Resolving huggingface.co (huggingface.co)... 3.165.160.12, 3.165.160.11, 3.165.160.59, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.12|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://us.gcp.cdn.hf.co/xet-bridge-us/67be35b066f702bfed7d3bdc/1d4f6eaf244defd216b981b498e9fba259ee17316f9595393c9bed1f126e3783?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27umt5_xxl_fp16.safetensors%3B+filename%3D%22umt5_xxl_fp16.safetensors%22%3B&Expires=1772041341&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiRXBvY2hUaW1lIjoxNzcyMDQxMzQxfX0sIlJlc291cmNlIjoiaHR0cHM6Ly91cy5nY3AuY2RuLmhmLmNvL3hldC1icmlkZ2UtdXMvNjdiZTM1YjA2NmY3MDJiZmVkN2QzYmRjLzFkNGY2ZWFmMjQ0ZGVmZDIxNmI5ODFiNDk4ZTlmYmEyNTllZTE3MzE2Zjk1OTUzOTNjOWJlZDFmMTI2ZTM3ODNcXD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&

## LIST MODELS

In [ ]:
!ls -al ./models/checkpoints/

total 6775444
drwxr-xr-x  2 root root       4096 Aug 30 23:40 .
drwxr-xr-x 22 root root       4096 Aug 30 23:39 ..
-rw-r--r--  1 root root 6938040682 Aug 30 23:40 cyberrealisticPony_v127Alt.safetensors
-rw-r--r--  1 root root          0 Aug 30 23:39 put_checkpoints_here


# `START ComfyUI  & Expose Server (MANUAL)`

## Download Prerequisits

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

In [ ]:
!python main.py --dont-print-server

# `START ComfyUI & Expose Server`

## Download Prerequisits

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

## CF Tunnel

In [17]:
import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2026-02-25 16:30:49.266
** Platform: Linux
** Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
** Python executable: /usr/bin/python3
** ComfyUI Path: /content/ComfyUI
** ComfyUI Base Folder Path: /content/ComfyUI
** User directory: /content/ComfyUI/user
** ComfyUI-Manager config path: /content/ComfyUI/user/__manager/config.ini
** Log path: /content/ComfyUI/user/comfyui.log

Prestartup times for custom nodes:
   0.0 seconds: /content/ComfyUI/custom_nodes/rgthree-comfy
   6.1 seconds: /content/ComfyUI/custom_nodes/ComfyUI-Manager

Found comfy_kitchen backend eager: {'available': True, 'disabled': False, 'unavailable_reason': None, 'capabilities': ['apply_rope', 'apply_rope1', 'dequantize_nvfp4', 'dequantize_per_tensor_fp8', 'quantize_nvfp4', 'quantize_per_tensor_fp8', 'scaled_mm_nvfp4']}
Found comfy_kitchen backend cuda: {'available': True, 'disabled': 

## localtunnel

In [ ]:
# localtunnel
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()
%cd /content/ComfyUI
!python main.py --dont-print-server